<a href="https://colab.research.google.com/github/lewis-m/yeast-mutation-tda/blob/master/PyStanTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Prerequesite installations
#Run once everytime runtime is restarted

import numpy as np
import cython

!pip install pystan

import pystan as ps

In [2]:
#Upload file/data
from google.colab import files
import pandas as pd

uploaded = files.upload()



Saving FirstData.xlsx to FirstData.xlsx


In [0]:
import numpy as np

xlsx = pd.ExcelFile('FirstData.xlsx')
df = pd.read_excel(xlsx, 'Wild Type')
df  = df.iloc[5:12]

t1 = df.iloc[:, [1,2,3]]
t2 = df.iloc[:, [6,7,8]]
t3 = df.iloc[:, [11,12,13]]
t4 = df.iloc[:, [16,17,18]]
t5 = df.iloc[:, [21,22,23]]

wild_type = np.array([t1.values, t2.values, t3.values, t4.values, t5.values])

df = pd.read_excel(xlsx, 'Y130C')
df  = df.iloc[5:12]

t1 = df.iloc[:, [1,2,3]]
t2 = df.iloc[:, [6,7,8]]
t3 = df.iloc[:, [11,12,13]]
t4 = df.iloc[:, [16,17,18]]
t5 = df.iloc[:, [21,22,23]]

y130c = np.array([t1.values, t2.values, t3.values, t4.values, t5.values])

df = pd.read_excel(xlsx, 'F53S')
df  = df.iloc[5:12]

t1 = df.iloc[:, [1,2,3]]
t2 = df.iloc[:, [6,7,8]]
t3 = df.iloc[:, [11,12,13]]
t4 = df.iloc[:, [16,17,18]]
t5 = df.iloc[:, [21,22,23]]

f53s = np.array([t1.values, t2.values, t3.values, t4.values, t5.values])

df = pd.read_excel(xlsx, 'E203K')
df  = df.iloc[5:12]

t1 = df.iloc[:, [1,2,3]]
t2 = df.iloc[:, [6,7,8]]
t3 = df.iloc[:, [11,12,13]]
t4 = df.iloc[:, [16,17,18]]
t5 = df.iloc[:, [21,22,23]]

e203k = np.array([t1.values, t2.values, t3.values, t4.values, t5.values])


#put all data into one data frame
df = np.array([wild_type, y130c, f53s, e203k])

In [0]:
#Caluculate variances
variance = np.var(df, axis = 1)

print(variance)

[[[0.407898743208211 0.44202692154055684 0.08872811066551656]
  [0.3482158047554182 0.2346684200252743 0.07985859685628442]
  [0.44241425823285035 0.20837829342483077 0.11903757620745647]
  [0.34025579141089596 0.09476706535235742 0.12930036536600711]
  [0.19981381767885548 0.02593906022502403 0.1449450950999056]
  [0.1488271854748052 0.24476519576693206 0.6979500950176968]
  [0.0436605479212564 0.02505282004636366 0.12271341234043029]]

 [[0.31274399999999997 0.07908000000000001 0.15630400000000003]
  [0.06798399999999998 0.122376 0.06270399999999998]
  [0.09893600000000002 0.162584 0.11512]
  [0.07109599999999999 0.18781600000000007 0.062104000000000006]
  [0.10921600000000001 0.09809599999999999 0.06509600000000001]
  [0.021416 0.033464 0.081344]
  [0.004136 0.0 0.004136000000000006]]

 [[0.5257999999999997 0.24490399999999993 0.17618400000000004]
  [0.08285600000000001 0.01981599999999999 0.088376]
  [0.025383999999999983 0.04598399999999999 0.06521600000000002]
  [0.05219999999999

In [37]:
#First STAN usage

#Maybe normalise automatically ??

time_stamps = [0.5, 2, 3.25, 3.75, 5, 10, 20]

yeast_code = """
data {
    int<lower=0> T; // number of trials
    int<lower=0> TS; // number of timesteps
    vector[TS] ts; // time values
    real S0[TS,T]; // param S0
    real S1[TS,T]; // param S1
    real S2[TS,T]; // param S2
    vector<lower=0>[3] sigma; // s.e. of effect estimates
}
parameters {
    real P1;
    real P2;
    real Pi;
}

model {
    for(i in 1:TS){
      S0[i,:] ~ normal(exp(-P1*ts[i]), sigma[1]);
      S1[i,:] ~ normal(P1*(1-Pi)/(P1-P2)*(exp(-P2*ts[i]) - exp(-P1*ts[i])), sigma[2]);
      S2[i,:] ~ normal(P1*Pi/(P1-P2)*(exp(-P2*ts[i]) - exp(-P2*ts[i])) + P2/(P1-P2)*(exp(-P1*ts[i]) - 1) + P1/(P1-P2)*(1-exp(-P2*ts[i])), sigma[3]);
    }
}
"""

S0 = np.stack([e203k[:, 0, 0], e203k[:, 1, 0], e203k[:, 2, 0], e203k[:, 3, 0], e203k[:, 4, 0], e203k[:, 5, 0], e203k[:, 6, 0]])
S1 = np.stack([e203k[:, 0, 1], e203k[:, 1, 1], e203k[:, 2, 1], e203k[:, 3, 1], e203k[:, 4, 1], e203k[:, 5, 1], e203k[:, 6, 1]])
S2 = np.stack([e203k[:, 0, 2], e203k[:, 1, 2], e203k[:, 2, 2], e203k[:, 3, 2], e203k[:, 4, 2], e203k[:, 5, 2], e203k[:, 6, 2]])

S0 = np.array(S0, dtype = float)
S1 = np.array(S1, dtype = float)
S2 = np.array(S2, dtype = float)

S0 = S0/5
S1 = S1/5
S2 = S2/5

yeast_dat = {'T': 5,
             'TS': 7,
             'ts': time_stamps,
             'S0': S0,
             'S1': S1,
             'S2': S2,
             'sigma': [1,1,1]}

sm = ps.StanModel(model_code=yeast_code)
fit = sm.sampling(data=yeast_dat, iter=1000, chains=4)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_9980d75c37ae0c796e2a8a8d8ec7483c NOW.


KeyboardInterrupt: ignored

In [35]:
print(fit)

Inference for Stan model: anon_model_9980d75c37ae0c796e2a8a8d8ec7483c.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
P1     0.21    0.16   0.24  -0.03-3.3e-3   0.09   0.41   0.66      2   2.88
P2   1.1e10   8.0e9 1.5e10  -1.43   -0.9  7.2e8 2.0e10 4.3e10      3   1.76
Pi  -5.2e10  3.7e10 6.8e10-2.3e11-8.7e10 -4.0e9   0.49   1.23      3   1.92
lp__ -164.1   18.11  25.64 -205.0 -190.3 -157.6 -141.4 -140.4      2  26.28

Samples were drawn using NUTS at Wed Apr 17 16:28:44 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).
